<a href="https://colab.research.google.com/github/tsherburne/de-textbook/blob/main/02_SCRE_Risk_Assessment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Initialize Notebook

In [ ]:
#@title Download & Install Notebook Packages

%pip install -q manb
import manb
env = manb.Environment('https://genesys.mission-aware.net', '/api/v1/')

In [ ]:
#@title Cloudflare Tunnel Authentication
env.Tunnel()

In [ ]:
#@title GENESY Login & Project Selection
env.Login()

# Operational Risk Assessment


In [ ]:
#@title Generate Control Structure
cs = manb.ControlStructure(env)

## Control Structure Diagram

A ***hierarchical control structure*** is a system 
model that is composed of feedback control loops. An effective 
control structure will enforce constraints on the behavior of the 
overall system.

----

### MBSE ***Control Structure*** Entities & Relationships

> <img src="https://raw.githubusercontent.com/tsherburne/de-textbook/main/images/mission-aware-mbse-control-structure.png" height="175" />

----

| Entity | Description |
| :---   | :----       | 
| Control Action | A controller provides control actions to control some process and to enforce constraints on the behavior of the <br> controlled process.|
| Component | A component is an abstract term that represents the physical or logical entity that performs a specific function <br> or functions.|
| Feedback | Process models may be updated in part by feedback used to observe the controlled process.|
| Function | A function is a transformation that accepts one or more inputs (items) and transforms them into outputs (items).|
| Item | An item represent flows within and between functions. An item is an input to or an output from a function.|
| Use Case | A use case identifies a piece of the functionality of a system in terms of how the users use the system to <br> achieve their goals (mission). Use cases are high-level descriptions of behavior under a specific set of conditions. <br>These high-level descriptions can then be elaborated to define the system behavior.|

----

In [ ]:
#@title System ***Context*** & ***Control Structure*** Diagram
cs.Diagram()

In [ ]:
#@title System ***Control Actions***
cs.ControlActionTable()

## STPA Losses, Hazards, & Hazardous Control Actions

--- 
### MBSE ***Operational Risk Assessment*** Entities & Relationships

> <img src="https://raw.githubusercontent.com/tsherburne/de-textbook/main/images/mission-aware-ma-mbse-risk-assesment.png" height="200" />

In [ ]:
#@title Initialize Risk Assessment
ra = manb.RiskAssessment(env, cs)

In [ ]:
#@title System ***Losses***
#@markdown Definition: A ***Loss*** involves something of value to stakeholders. 
#@markdown Losses may include a loss of human life or human injury, property damage, 
#@markdown environmental pollution, loss of mission, loss of reputation, loss or leak of 
#@markdown sensitive information, or any other loss that is unacceptable to the stakeholders.

ra.LossTable()

In [ ]:
#@title System Hazards
#@markdown Definition: A ***Hazard*** is a system state or set of conditions 
#@markdown that, together with a particular set of worst-case environmental 
#@markdown conditions, will lead to a loss.

ra.HazardTable()

In [ ]:
#@title System Hazardous Control Actions
#@markdown Definition: A ***Hazardous Control Action (HCA)*** is a control action that, 
#@markdown in a particular context and worst-case environment, will lead to a hazard.
#@markdown 
#@markdown There are four ways ***(variation type)*** a control action can be hazardous:
#@markdown 1. Not providing the control action leads to a hazard.
#@markdown 2. Providing the control action leads to a hazard.
#@markdown 3. Providing a potentially safe control action but too early, too late, or in the wrong order
#@markdown 4. The control action lasts too long or is stopped too soon (for continuous control actions, not discrete ones).
#@markdown > To filter rows NOT containing \<string>
#@markdown ```
#@markdown ^((?!<string>).)*$
#@markdown ```

ra.HazardousActionTable()

In [ ]:
#@title Control Action Analysis

#@markdown Although all four variation types should be considered, they may not all 
#@markdown be applicable in every case (see HCA.0 and 'justification'). It is also possible 
#@markdown to identify several HCAs for a single variation type.
#@markdown In general, each CA / variation type may define 0 or more HCAs.

ra.ControlActionAnalysisTable()

# Exercises

In [ ]:
#@title Risk Assessment Questions
ex = manb.Exercises(env, manb.Section.RISK_ASSESSMENT)
ex.Edit()